# Working with HEALPix Data

In this notebook, we showcase how UXarray can be used with HEALPix data.
* A brief overview into HEALPix
* Representing a HEALPix grid in the UGRID conventions
* Loading data that resides on HEALPix Grids

In [ ]:
import cartopy.crs as ccrs

import uxarray as ux

## What is HEALPix?

The Hierarchical Equal Area isoLatitude Pixelisation (HEALPix) algorithm is a method for the pixelisation of the
2-sphere. It has three defining qualities.
- The sphere is hierarchically tessellated into curvilinear quadrilaterals
- Areas of all pixels at a given resolution are identical
- Pixels are distributed on lines of constant latitude

```{note}
For more information on HEALPix, you can refer to [this excellent overview](https://easy.gems.dkrz.de/Processing/healpix/index.html#hierarchical-healpix-output) in DKRZ's Easy Gems documentation.
```






## Representing a HEALPix Grid in the UGRID Conventions

Most datasets loaded into **UXarray** come with a dedicated grid file. However, for HEALPix datasets, the underlying grid structure is inferred from the number of cells.

For a HEALPix grid in a nested layout, the number of cells is defined by:

$$
N_{\text{cells}} = 12 \cdot z^4,
$$

where `z` is the *zoom level*. A higher zoom level increases the effective resolution of the grid.

The ``ux.Grid.from_healpix(zoom)`` classmethod can be used to construct a ``ux.Grid`` instance representing a HEALPix grid in the UGRID conventions.





In [ ]:
ux.Grid.from_healpix(zoom=3)

By default, our ``Grid`` will only contain the face coordinate values, which represent the HEALPix pixel locations. We can see this above with only the ``face_lon`` and ``face_lat`` variables populated.

However, much of UXarray functionality requires the boundaries of each cell to be defined. This is represented using the following variables:
- ``node_lon``: Longitude of the corners of each cell
- ``node_lat``: Latitude of the corners of each cell
- ``face_node_connectivity``: The indices of the nodes that surround each face

An optional parameter, ``pixels_only``, is provided which can be set to ``True`` if you require the boundaries to be computed upfront.

In [ ]:
ux.Grid.from_healpix(zoom=3, pixels_only=False)

However, even if you load in a HEALPix grid specifying that you do not want the connectivity upfront, they can still be constructed when desired because of UXarray's internal design.

In [ ]:
ux.Grid.from_healpix(zoom=3, pixels_only=True).face_node_connectivity

### Grid-Agnostic Functionality

Once loaded in, UXarray does not care about the underlying grid format, as it represents all formats in its unified UGRID-like convention. This means that all existing functionality is supported through the same UXarray inferface. Below are basic examples of plotting & remapping.

#### Plotting

By using UXarray's plotting functionality, we can observe how increasing the ``zoom`` level leads to a higher-resolution grid.



In [ ]:
(
    ux.Grid.from_healpix(zoom=2).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=2"
    )
    + ux.Grid.from_healpix(zoom=3).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=3"
    )
    + ux.Grid.from_healpix(zoom=4).plot(
        periodic_elements="split", projection=ccrs.Orthographic(), title="zoom=4"
    )
).cols(1)

#### Remapping

Below is a simple example of remapping UGRID data residing on a ``CSne30`` grid to a HEALPix grid with a zoom level of 5.

In [ ]:
source_uxds = ux.open_dataset(
    "../../test/meshfiles/ugrid/outCSne30/outCSne30.ug",
    "../../test/meshfiles/ugrid/outCSne30/outCSne30_vortex.nc",
)

# source data & grid
source_uxds["psi"].plot(
    cmap="inferno", projection=ccrs.Orthographic(), title="Source Data"
)

# destination grid
hp_grid = ux.Grid.from_healpix(zoom=5)

Before remapping, we can plot our Source and Destination grids.

In [ ]:
source_uxds.uxgrid.plot(
    projection=ccrs.Orthographic(), title="Source Grid (CSne30)"
) + hp_grid.plot(projection=ccrs.Orthographic(), title="Destination Grid (HEALPix)")

We can now perform our remapping. In this case, we apply a simple nearest neighbor remapping.

In [ ]:
psi_hp = source_uxds["psi"].remap.nearest_neighbor(destination_grid=hp_grid)
psi_hp

Our original data variable now resides on our HEALPix grid.

In [ ]:
psi_hp.plot(cmap="inferno", projection=ccrs.Orthographic(), title="Remapped Data")

We can now chain together a few functions to convert our output from a UXarray Dataset to a NetCDF file, with the grid represented in the HEALPix format.
- `to_dataset()`: Converts the ``ux.UxDataArray`` to a ``ux.UxDataset``
- `to_xarray(grid_format="HEALPix")`: Converts our ``ux.UxDataset`` to a ``xr.Dataset`` encoded in the HEALPix format.
- `to_netcdf("psi_healpix.nc")`: Saves our dataset to disk as a NetCDF file.

In [ ]:
psi_hp.to_dataset().to_xarray(grid_format="HEALPix").to_netcdf("psi_healpix.nc")

## HEALPix Equal Area Property

HEALPix's fundamental property is that **all pixels at a given resolution have exactly the same spherical area**. This "equal area" property is crucial for scientific applications such as global averaging, zonal means, and conservation properties in regridding operations.

### Geometric Representation Considerations

UXarray represents HEALPix grids using Great Circle Arcs (GCAs) to define pixel boundaries, following UGRID conventions. However, this geometric representation can introduce systematic errors when computing areas numerically, potentially violating HEALPix's equal-area property. 

```{note}
Don't worry about the systematic differences between UXarray and HEALPix as we fulfill the healpix equal area property in our `Grid.face_areas` property, making sure that all the faces in a healpix mesh have the same theoretical HEALPix area.
```

Let's demonstrate this with HEALPix's 12 base pixels:

In [ ]:
import numpy as np

# Create HEALPix grid at resolution 0 (12 base pixels)
grid = ux.Grid.from_healpix(0, pixels_only=False)

# Using face_areas property ensures equal areas for HEALPix
hp_areas = grid.face_areas.values
print(f"Standard deviation: {np.std(hp_areas):.2e}")
print(f"All pixels have area: {hp_areas[0]:.6f} steradians")

### Still need geometric face area calculations for a HEALPix Mesh?

For most use cases, the `Grid.face_areas` property provides the recommended approach for accessing face areas. However, if you specifically need geometric calculations of individual HEALPix faces as they are represented in UXarray (rather than the theoretical equal areas), you can access the internal computation method. Note that this approach may not preserve HEALPix's equal-area property due to geometric representation differences. 

In [ ]:
# For advanced use cases: access geometric calculations (may not preserve equal-area property)
hp_geometric_areas, face_jacobians = grid._compute_face_areas(
    quadrature_rule="triangular", order=4
)
print(
    f"Geometric std deviation: {hp_geometric_areas.std():.2e} (vs theoretical equal areas)"
)

If you are interested in futher details of the systematic errors due to geometric calculation with a clear spatial pattern. Our analysis across resolution levels 0-7 shows:

- **Equatorial pixels** (lat≈0°): +20.5% area error
- **Mid-latitude pixels** (lat≈±42°): -9.9% area error  
- **Maximum errors**: Persist at ~10% even at fine resolutions


## Loading HEALPix Data


In the remapping example above, we successfully remapped a data variable onto a HEALPix grid and saved it as a NetCDF file.

UXarray extends Xarray's ``Dataset`` and ``DataArray`` classes to provide grid-aware implementations.





Using the example we generated above, we can load in data that is stored in the HEALPix format using the ``UxDataset.from_healpix()`` classmethod.

In [ ]:
uxds = ux.UxDataset.from_healpix("psi_healpix.nc")
uxds

The interface above looks almost identical to what you would see if you loaded in the file directly with Xarray.

In [ ]:
import xarray as xr

xr.open_dataset("psi_healpix.nc")

However, there are a few noticeable differences and additions that streamline UXarray's implementation and provide more context on what grid the data resides on:
1. The "Show Grid Information" tab provides information on the underlying ``Grid`` object that is attached via the ``.uxgrid`` attribute
2. The ``zoom`` for HEALPix grids is automatically determined from the ``cells`` dimension, with an appropriate ``Grid`` constructed from that
3The dimensions are renamed to match the UGRID conventions, with ``cells`` being renamed to ``n_face`` for the case of HEALPix grids

We can then directly access our ``Grid`` using the ``.uxgrid`` attribute, allowing for grid-specific functionallity as described in the sections above.




In [ ]:
uxds.uxgrid

## Future Work

Support for the HEALPix format is still in its early stages. We welcome your suggestions and contributions! To get started, please check out our [Contributors Guide](https://uxarray.readthedocs.io/en/stable/contributing.html).
